# Chapter 5: First-Class Functions

Functions in Python are first-class objects, meaning they can be

* created at runtime,

* assigned to a variable or element in a data structure,

* passed as an argument to a function,

* returned as the result of a function.

Having first-class functions enables programming in a functional style.

## Treating a Function Like an Object

The cells below show that `factorial` is an instance of the `function` class, e.g. we can read its `__doc__` attribute.

In [1]:
def factorial(n):
    """Returns n!"""
    return 1 if n < 2 else n * factorial(n - 1)

In [2]:
factorial.__doc__

'Returns n!'

In [7]:
print(type(factorial))

<class 'function'>


In [8]:
fact = factorial
fact(5)

120

## Higher-Order Functions

A function that takes a function as an argument or returns a function as a result is called a *higher-order function*, e.g. `sorted` which allows us to sort values using a one-argument function.

Some of the most well-known built-in higher-order functions are `map`, `reduce`, `filter` and `apply`. The latter is deprecated, the three preceding functions are somewhat obsolete due to the introduction of listcomps.

In [11]:
# These two are equivalent
maplist = list(map(factorial, range(5)))
listcomp = [factorial(i) for i in range(5)]

print(maplist)
print(listcomp)

[1, 1, 2, 6, 24]
[1, 1, 2, 6, 24]


`map` and `filter`return generators, so their direct substitue is a generator expression.

In [12]:
g1 = map(factorial, range(5))
g2 = (factorial(i) for i in range(5))

print(next(g1))
print(next(g2))

1
1


The `reduce` function was demoted from built-in to the `functools` module, and is largely replaced by the use of conditionals in listcomps. It's most common use, summation, has largely been replaced by the `sum` built-in.

In [15]:
from functools import reduce
from operator import add

reduce(add, range(5)) == sum(range(5))

True

Other reducing built-ins are `any(iterable)` and `all(iterable)`.

`any` returns `True` if any element of `iterable` is truthy.

`all` returns `True` if every element of `iterable` is truthy.

## Anonymous Functions

Anonymous functions, or `lambda` functions, are handy when we need to create small one.-off functions.

The body of a `lambda` cannot make assignents or use any other Python statements, such as `while`, `try`, etc.

The best use of an anonymous function is in the context of an argument list.

In [17]:
# Returns list sorted by reversed spelling
fruits = ["apple", "banana", "cherry", "raspberry", "strawberry", "fig"]
sorted(fruits, key=lambda word: word[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

### Lundh's Lambda Refactoring Recipe

If an anonymous function is hard to understand:

1. Write a comment explaining what `lambda` does.

2. Think of a name that captures the essence of the `lambda`.

3. Convert `lambda` to a `def` statement using that name.

4. Remove the comment.